## Importing Libraries


In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import string
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import random
from sklearn.preprocessing import LabelEncoder

## Content loading
Content is a JSON file containing tag, intent and response.

In [4]:
import json
file_path = "/content/drive/MyDrive/DarijaChatbotData.json"
with open(file_path) as DarijaChatbotData:
    intents = json.load(DarijaChatbotData)


In [5]:
intents

{'intents': [{'tag': 'Greetings',
   'patterns': ['Salam',
    'Fen',
    'Salamalaykom',
    'Wach',
    'Fen had lghbour',
    'Bonjour cv'],
   'responses': ['Salam! Kifach bghiti n3awnek lyoom?'],
   'context': ['']},
  {'tag': 'CourtesyGreeting',
   'patterns': ['Kif dayer?',
    'Salam, kif dayer lyoom?',
    'Salamalaykom, kif dayer lyoom?',
    'Fen, cv?',
    'Ach kat 3awd, kif dayer?',
    'Bikhir?',
    'Rak mzyaan?',
    'Natmanak tkkon bikhir?',
    'Nta bikhir?',
    'Labas?',
    'Kifach kayn?',
    'Wach kayn noved?',
    'Kif dayer?',
    'Kifach had lyoum?'],
   'responses': ['Ana labas, w nta kifach?',
    'Salam, ana labas. W nta?',
    'Wa 3alaykom salam, ana mzyaan. Shnu ljadid?',
    'Ana hamdellah mzyaan, shukran. W nta?',
    'Ana bikhir, choukran. W nta?',
    'Labas, shukran. Nta bikhir?'],
   'context': ['']},
  {'tag': 'RealNameQuery',
   'patterns': ['chno smitek?',
    'ckon nta?',
    'ch7al f3amrek?',
    'minin nta',
    'mnin nta?'],
   'responses': [

In [6]:
tags = []
patterns = []
responses = {}
for intent in intents['intents']:
    responses[intent['tag']] = intent["responses"]
    for lines in intent['patterns']:
        patterns.append(lines)
        tags.append(intent['tag'])


print("this is tags",tags)
print("this is input",patterns)
print("this is responses",responses)

this is tags ['Greetings', 'Greetings', 'Greetings', 'Greetings', 'Greetings', 'Greetings', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'CourtesyGreeting', 'RealNameQuery', 'RealNameQuery', 'RealNameQuery', 'RealNameQuery', 'RealNameQuery', 'RealName', 'NameQuery', 'Moro', 'Semaine', 'Mois', 'Jokes', 'Jokes', 'IA', 'GoodBye', 'GoodBye', 'GoodBye', 'GoodBye', 'GoodBye']
this is input ['Salam', 'Fen', 'Salamalaykom', 'Wach', 'Fen had lghbour', 'Bonjour cv', 'Kif dayer?', 'Salam, kif dayer lyoom?', 'Salamalaykom, kif dayer lyoom?', 'Fen, cv?', 'Ach kat 3awd, kif dayer?', 'Bikhir?', 'Rak mzyaan?', 'Natmanak tkkon bikhir?', 'Nta bikhir?', 'Labas?', 'Kifach kayn?', 'Wach kayn noved?', 'Kif dayer?', 'Kifach had lyoum?', 'chno smitek?', 'ckon nta?', 'ch7al f3amrek?', 'minin nta', '

## Data Preprocessing
Now we construct a dataframe consist of patterns and their respective tags.

In [7]:
data = pd.DataFrame({"inputs":patterns,
                    "tags":tags})

In [8]:
data

,inputs,tags
0,Salam,Greetings
1,Fen,Greetings
2,Salamalaykom,Greetings
3,Wach,Greetings
4,Fen had lghbour,Greetings
5,Bonjour cv,Greetings
6,Kif dayer?,CourtesyGreeting
7,"Salam, kif dayer lyoom?",CourtesyGreeting
8,"Salamalaykom, kif dayer lyoom?",CourtesyGreeting
9,"Fen, cv?",CourtesyGreeting


## Tokenizing & Padding

In [9]:
tokenizer = Tokenizer(num_words = 2000)

In [10]:
tokenizer.fit_on_texts(data["inputs"])

In [11]:
train = tokenizer.texts_to_sequences(data["inputs"])

In [12]:
train

[[8],
 [4],
 [9],
 [5],
 [4, 10, 21],
 [22, 11],
 [1, 2],
 [8, 1, 2, 12],
 [9, 1, 2, 12],
 [4, 11],
 [13, 14, 23, 1, 2],
 [6],
 [24, 25],
 [26, 27, 6],
 [3, 6],
 [28],
 [15, 16],
 [5, 16, 29],
 [1, 2],
 [15, 10, 30],
 [31, 32],
 [33, 3],
 [34, 35],
 [36, 3],
 [37, 3],
 [38, 39],
 [40, 41, 7, 42, 43, 44],
 [45, 46, 47, 48, 49, 50, 51, 52],
 [17, 7, 53, 18, 54],
 [13, 55, 56, 18, 57],
 [17, 7, 58, 19],
 [59, 19, 60],
 [5, 14, 61, 62],
 [63, 64, 65],
 [20, 20],
 [66],
 [67, 68],
 [69]]

In [13]:
x_train = pad_sequences(train)

In [14]:
x_train

array([[ 0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  0,  4, 10, 21],
       [ 0,  0,  0,  0,  0,  0, 22, 11],
       [ 0,  0,  0,  0,  0,  0,  1,  2],
       [ 0,  0,  0,  0,  8,  1,  2, 12],
       [ 0,  0,  0,  0,  9,  1,  2, 12],
       [ 0,  0,  0,  0,  0,  0,  4, 11],
       [ 0,  0,  0, 13, 14, 23,  1,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  6],
       [ 0,  0,  0,  0,  0,  0, 24, 25],
       [ 0,  0,  0,  0,  0, 26, 27,  6],
       [ 0,  0,  0,  0,  0,  0,  3,  6],
       [ 0,  0,  0,  0,  0,  0,  0, 28],
       [ 0,  0,  0,  0,  0,  0, 15, 16],
       [ 0,  0,  0,  0,  0,  5, 16, 29],
       [ 0,  0,  0,  0,  0,  0,  1,  2],
       [ 0,  0,  0,  0,  0, 15, 10, 30],
       [ 0,  0,  0,  0,  0,  0, 31, 32],
       [ 0,  0,  0,  0,  0,  0, 33,  3],
       [ 0,  0,  0,  0,  0,  0, 34, 35],
       [ 0,  0,  0,  0,  0,  0, 36,  3],
       [ 0,  0, 

## Encoding the output

In [ ]:
le = LabelEncoder()

In [ ]:
y_train = le.fit_transform(data["tags"])

In [ ]:
y_train

array([ 9,  9,  9,  9,  9,  9,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  5,  5,  5,  5,  5,  5,  5, 19, 19, 19, 19, 19, 19, 19,
        5,  5,  5,  5,  5,  5,  5, 15, 15, 15, 15, 15, 15, 22, 22, 22, 22,
       22, 22, 22, 21, 21, 21, 21, 21, 21, 17, 17, 17, 17, 17, 17, 17, 12,
       12, 12, 12, 12, 24, 24, 24, 24, 24, 24,  8,  8,  8,  8,  8,  8,  3,
        3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,
        1,  1, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 18, 18

In [15]:
input_shape = x_train.shape[1]
print(input_shape)

8


In [22]:
le = LabelEncoder()
y_train = le.fit_transform(data["tags"])
input_shape = x_train.shape[1]
unique_words = len(tokenizer.word_index)
output_length = le.classes_.shape[0]
print("Number of unique words: ", unique_words)
print("Output size: ", output_length)

Number of unique words:  69
Output size:  11


## Constructing a Neural Network
La couche initiale est une couche d'intégration, facilitant la transformation des jetons d'entrée en vecteurs denses de taille fixe. Par la suite, deux couches de mémoire bidirectionnelle à long et à court terme (LSTM) sont configurées pour renvoyer des séquences. Une couche Dropout suit, fournissant une régularisation pour éviter le surajustement. Le modèle aplatit ensuite la sortie et la fait passer à travers une couche dense. Enfin, la couche de sortie se compose d'une couche Dense avec des unités output_length et une fonction d'activation softmax.

In [23]:
model = tf.keras.Sequential()
model.add(Embedding(unique_words + 1, 50, input_length=input_shape))
model.add(Bidirectional(LSTM(10, return_sequences=True)))
model.add(Bidirectional(LSTM(10, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=output_length, activation='softmax'))

In [24]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer ='adam', metrics=['accuracy'])

## Training the Model
Here, I am train the neural network. The training is performed on a dataset for 400 epochs.

In [25]:
MoroccanBot = model.fit(x_train, y_train, epochs=400)

Epoch 1/400
2/2 [==============================] - 9s 39ms/step - loss: 2.3991 - accuracy: 0.0000e+00
Epoch 2/400
2/2 [==============================] - 0s 19ms/step - loss: 2.3954 - accuracy: 0.1579
Epoch 3/400
2/2 [==============================] - 0s 19ms/step - loss: 2.3928 - accuracy: 0.2368
Epoch 4/400
2/2 [==============================] - 0s 18ms/step - loss: 2.3900 - accuracy: 0.3684
Epoch 5/400
2/2 [==============================] - 0s 19ms/step - loss: 2.3843 - accuracy: 0.3684
Epoch 6/400
2/2 [==============================] - 0s 18ms/step - loss: 2.3807 - accuracy: 0.3684
Epoch 7/400
2/2 [==============================] - 0s 21ms/step - loss: 2.3787 - accuracy: 0.3684
Epoch 8/400
2/2 [==============================] - 0s 19ms/step - loss: 2.3734 - accuracy: 0.3684
Epoch 9/400
2/2 [==============================] - 0s 18ms/step - loss: 2.3693 - accuracy: 0.3684
Epoch 10/400
2/2 [==============================] - 0s 19ms/step - loss: 2.3659 - accuracy: 0.3684
Epoch 11/400
2/

In [26]:
print("Accuracy: ",MoroccanBot.history['accuracy'][-1])

Accuracy:  1.0


## Chatting with bot

In [27]:

print("MoroccanBot: Salam ! Ana chatbot maghribi । kifach bghiti n3awnek ?")
while True:
    textList = []
    USER  = input("badr: ")
    prediction_input = []


    prediction_input = ''.join(USER )
    textList.append(prediction_input)

    prediction_input = tokenizer.texts_to_sequences(textList)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    output = model.predict(prediction_input)
    output = output.argmax()

    response_tag = le.inverse_transform([output])[0]
    print("MoroccanBot: ", random.choice(responses[response_tag]))
    if response_tag == 'GoodBye':
        break

MoroccanBot: Salam ! Ana chatbot maghribi । kifach bghiti n3awnek ?
badr: salam
1/1 [==============================] - 2s 2s/step
MoroccanBot:  Salam! Kifach bghiti n3awnek lyoom?
badr: Kif dayer?
1/1 [==============================] - 0s 39ms/step
MoroccanBot:  Labas, shukran. Nta bikhir?
badr: chno smitek?
1/1 [==============================] - 0s 26ms/step
MoroccanBot:  ana chatBOT mn a9dam o a3ra9 balad fl3aalaam lmaghrib, fach bghiti n3awnek lyoom
badr: Chkon saybek?
1/1 [==============================] - 0s 22ms/step
MoroccanBot:  Sayabni BADREDDINE AIT EL KHADIR o howa li 3alamnii
badr: momkin tktab lia b 7orof 3arabiya
1/1 [==============================] - 0s 26ms/step
MoroccanBot:  نعم فاش بغيتي نعاونك اليوم
badr: ممكن تقول لي شي حاجة على تاريخ المغرب
1/1 [==============================] - 0s 23ms/step
MoroccanBot:  المغرب هو بلاد ذات جذور عميقة في التاريخ، والاكتشافات الأخيرة تبثت عمق هذه الجذور لي تمتد إلى العصور القديمة. ثراء الحضارة المغربية كاي رجع لآلاف السنين، المغرب ش